In [33]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import numpy as np
from sklearn.metrics import f1_score
import time

from sklearn.model_selection import train_test_split

import sys
sys.path.append('../')
from helper import *

runs = pd.read_pickle("../Data/main_1.df")
FEATURES = ['actual_weight','declared_weight','draw','win_odds',"jockey_record",
'horse_record','distance', 'trainer_record']
TARGET = "won"
num_races = len(np.unique(runs["race_id"]))
X = runs[FEATURES]
y = runs[TARGET]

testPct = 0.2
valPct = 0.2
trainIndex = int(num_races * (1-(testPct+valPct)))
valIndex =  int(num_races * (1-valPct))
train_race_id = np.unique(runs["race_id"])[trainIndex]
val_race_id = np.unique(runs["race_id"])[trainIndex]
X_train = X.loc[runs["race_id"]<=train_race_id]
y_train = y.loc[runs["race_id"]<=train_race_id]
X_test = X.loc[runs["race_id"]>train_race_id]
y_test = y.loc[runs["race_id"]>train_race_id]
race_sizes_for_eval = [len(runs.loc[runs["race_id"]==race_id]["race_id"]) for race_id in np.unique(runs.loc[runs["race_id"]>max_race_id]["race_id"])]

In [34]:
nb_model = GaussianNB()
kfold = KFold(n_splits = 5)
score_nb_Win=cross_val_score(nb_model,X_train,y_train,cv=kfold,scoring='f1')

In [35]:
nb_model.fit(X_train, y_train)
preds = nb_model.predict(X_test)

In [36]:
meanScore, stdScore = crossVal(runs, FEATURES, TARGET, nb_model)

CV 1/4
CV 2/4
CV 3/4
CV 4/4
Mean score: 0.213 +/- 0.016


In [37]:
champion_index = y_test[y_test == 1].index.tolist()
champion_odds = X_test[y_test == 1]['win_odds'].tolist()

In [38]:
def count_range_in_list(li, min, max):
    ctr = 0
    for x in li:
        if min <= x <= max:
            ctr += 1
    return ctr

def ele_in_list(li, min, max):
    ele = []
    for x in li:
        if min <= x <= max:
            ele.append(x)
    return ele

def betting_result(champion_odds,champion_index,prediction):
    money=0
    for i in range(len(champion_index)-1):
        ctr= count_range_in_list(prediction,champion_index[i],champion_index[i+1]-1)
        if ctr==0:
            money=money-1
        elif ctr==1:
            money=money-1+champion_odds[i]
        else:
            ele_list=ele_in_list(prediction,champion_index[i],champion_index[i+1]-1)
            if min(ele_list)==champion_index[i]:
                money=money-1+champion_odds[i]
            else:
                money=money-1
    ctr = count_range_in_list(prediction, champion_index[len(champion_index)-1],len(y_test)-1)
    if ctr == 0:
        money = money - 1
    elif ctr == 1:
        money = money - 1 + champion_odds[len(champion_index)-1]
    else:
        ele_list = ele_in_list(prediction, champion_index[len(champion_index)-1], len(testing['HorseWin'])-1)
        if min(ele_list)==champion_index[len(champion_index)-1]:
            money = money - 1 + champion_odds[len(champion_index)-1]
        else:
            money = money - 1
    return money

In [39]:
preds = pd.DataFrame(preds)
preds_i = preds[preds == 1].index.tolist()
betting_result(champion_odds, champion_index, preds)

-2546

In [40]:
score_nb_Win=cross_val_score(nb_model,X_train,y_train,cv=kfold,scoring='f1')
print("Cross Validation score for Naive Bayes: ",'\n', "HorseWin: ",score_nb_Win)

Cross Validation score for Naive Bayes:  
 HorseWin:  [0.18493151 0.18167331 0.18765432 0.17269737 0.14589104]


In [42]:
f1_score(preds, y_test)

0.1857482185273159

In [45]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
score_nb_Win=cross_val_score(lr,X_train,y_train,cv=kfold,scoring='f1')

In [46]:
print(score_nb_Win)

[0. 0. 0. 0. 0.]
